In [13]:
%pwd

'd:\\Linh TInh\\Sang RD\\FL-DP'

In [20]:
from tek4fed import data_lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk.corpus import stopwords
from textblob import Word
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
import pandas as pd
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


seed = 2023
torch.manual_seed(seed)


def cleaning_smsspam_dataset(df_data):
    # covert uppercase letters to lowercase letters
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        x.lower() for x in x.split()
    ))

    # delete puctuation marks
    df_data['text'] = df_data['text'].str.replace('[^\w\s]', '')

    # delete numbers from texts
    df_data['text'] = df_data['text'].str.replace('\d', '')

    # delete stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = list(string.punctuation)
    stop_words.update(punctuation)
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        x for x in x.split() if x not in stop_words
    ))

    # lemmatization and get the roots of the words
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        [Word(word).lemmatize() for word in x.split()]
    ))

    # remove words less than 3 letters
    df_data['text'] = df_data['text'].apply(lambda x: ' '.join(
        [x for x in x.split() if len(x) > 3]
    ))

    return df_data


def preprocessing_smsspam_dataset(datafile):
    # load dataset
    data = pd.read_csv(datafile, encoding='ISO-8859-1', 
                   engine='python')
    #rename dataset columns
    data.rename(columns = {"v1": "target", "v2": "text"}, inplace = True)

    #drop unnecessary columns
    data.drop(["Unnamed: 2","Unnamed: 3", "Unnamed: 4"], axis = 1, inplace = True)

    # drop duplicate data
    data.drop_duplicates(inplace = True)

    # cleaning data
    data = cleaning_smsspam_dataset(data)

    data['target'].replace({'ham': 0, 'spam': 1}, inplace=True)
    x_data = data['text'].values
    y_data = data['target'].values

    tokenizer = get_tokenizer(tokenizer='spacy', language='en_core_web_trf')
    tokens = [tokenizer(doc) for doc in x_data]
    
    vocab = build_vocab_from_iterator(tokens)
    # label = torchtext.legacy.data.LabelField(dtype = torch.float, batch_first=True)
    # fields = [("type", label),('text', text)]

    # training_data = data.TabularDataset(path=data_file, format="csv", fields=fields, skip_header=True)
    # print(dir(training_data))

    print(len(vocab))

    return data

def smsspam_load_data(test_prob):
    data_file = "./datasets/smsspam/spam.csv"
    preprocessing_smsspam_dataset(data_file)
    # print(dir(pre))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
smsspam_load_data(0.5)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12372\1924160923.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df_data['text'] = df_data['text'].str.replace('[^\w\s]', '')
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12372\1924160923.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  df_data['text'] = df_data['text'].str.replace('\d', '')


6972
